In [2]:
import pandas as pd

In [4]:
# Packages
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.error import HTTPError
from googletrans import Translator


#Pages Changing Url_Values 
values = [1, 51, 101, 151, 201]

#fileName CSV File
filename = "IMDB.csv"

#Opening 
f = open(filename, "w")


#Headers Used as Columns
headers = "ID, name, year, certificate, Genre, rating, metascore, Discription, Director, stars, Gross\n"
f.write(headers)

#Looping Through All Pages
for i in values:
    if i == 1:
        url_In = 'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating'
    else:
        url_In = 'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start={}&ref_=adv_nxt'.format(
            i)
    #List Of All Urls
    list_url = [url_In.format(i) for i in values]


    
    # OPENING UP CONNECTION ,GRABBING DATA
    try:
        uClient = uReq(url_In)
    except HTTPError as e:
        print(e)
    except URLError as e:
        print('The Server Not Found')
    else:
        page_html = uClient.read()
        uClient.close()

    # HTML PARSING
    page_soup = soup(page_html.decode('utf-8','ignore'), "html.parser")

    # containers
    containers = page_soup.findAll("div", {"class": "lister-item mode-advanced"})


    # LOPPING in Each containers of Cards

    for item in containers:
        item_content = item.find('div', {'class': "lister-item-content"})
        # index
        ID = int(item_content.h3.span.text.split('.')[0])

        # Movie
        #name = item_content.h3.a.text
        Uk_Name = item_content.h3.a.string

        #using google API Translator
        tr = Translator()
        name = tr.translate(Uk_Name).text
        print(name)


        # year
        try:
            year = int(item_content.h3.find('span', {'class': "lister-item-year text-muted unbold"}).text[1:5])
        except ValueError:
            year = int(item_content.h3.find('span', {'class': "lister-item-year text-muted unbold"}).text[5:9])

   # Other colmns Data for gener , certificate , Time
        block1 = item_content.find('p', {'class': 'text-muted'})

        # certificate
        if block1.find('span', {'class': "certificate"}):
            certificate = block1.find('span', {'class': "certificate"}).text
        else:
            certificate = None

        # Genre
        Genre = block1.find('span', {'class': "genre"}).text.replace('\n', '')

        # BLOCK FOR RATING METASCORE
        block2 = item_content.find('div', {'class': 'ratings-bar'})
        
        # Rating
        rating = float(block2.div.strong.text)
        
        # Metascore
        if block2.find(class_='inline-block ratings-metascore'):
            metascore = int(block2.find('div', {'class': 'inline-block ratings-metascore'}).span.text)
        else:
            metascore = None
        
        # Description
        Discription = (item_content.findAll('p', {'class': 'text-muted'})[1]).text.replace('\n', '')

        # Director
        block3 = item_content.find('p', {'class': ""}).find_all('a')
        Director = block3[0].text.replace(',','|')

        # stars
        stars = ",".join(block3[i].text for i in range(1, len(block3)))
        
        # Gross
        try:
            Gross = (item_content.find('p', {'class': "sort-num_votes-visible"}).findAll('span')[4]).text
        except IndexError:
            Gross =None
        

        f.write(
            '{},{},{},{},{},{},{},{},{},{},{}\n'.format(ID, name.replace(',', '.'), year, certificate,
                                                     Genre.replace(',', '|'),
                                                     rating, metascore, Discription.replace(',', '.'), Director,
                                                     stars.replace(',', ' |'),Gross))
                                                     
f.close()



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
data = pd.read_csv('IMDB.csv')

In [7]:
data.head()

,ID,name,year,certificate,Genre,rating,metascore,Discription,Director,stars,Gross
